In [27]:
### Importeer bibliotheken en utils

import os
import pandas as pd
import json
from IPython.display import JSON as JSONDisplay
import requests
from re import sub
import ast
import re

import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Sequence
from sqlalchemy.dialects.postgresql import insert
import uuid

def generateEAUUID():
    return '{' +  str(uuid.uuid4()).upper() + '}'

# Wijzig de maximale breedte van kolommen (bijvoorbeeld naar 100 tekens)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_rows', 20) 

# Controleer of de instelling is toegepast
print(pd.get_option('max_colwidth'))

generateEAUUID()

100


'{51BA1AEB-382E-481F-B741-AB1522EEA35E}'

In [28]:
### Configuratie
db_uri = "sqlite:////home/jovyan/root/v2.2.0-develop/gemeentelijk gegevensmodel EA16.qea"


In [33]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.automap import automap_base

# Databaseverbinding
engine = create_engine(db_uri)

# Reflecteer metadata
metadata = MetaData()
metadata.reflect(bind=engine)
print("Beschikbare tabellen:", metadata.tables.keys())

# Automap Base instellen
Base = automap_base(metadata=metadata)
Base.prepare(autoload_with=engine)

# Controleer beschikbare klassen
print("Beschikbare klassen:", Base.classes.keys())

# Toegang tot klassen (gebruik de juiste tabelnamen)
try:
    Object = Base.classes.t_object
    XRef = Base.classes.t_xref
    ObjProp = Base.classes.t_objectproperties
    print("Klassen geladen succesvol.")
except AttributeError as e:
    print("Fout bij het laden van klassen:", e)
    

Beschikbare tabellen: dict_keys(['sqlite_sequence', 't_attribute', 't_attributeconstraints', 't_attributetag', 't_authors', 't_cardinality', 't_category', 't_clients', 't_complexitytypes', 't_connector', 't_connectorconstraint', 't_connectortag', 't_connectortypes', 't_constants', 't_constrainttypes', 't_datatypes', 't_diagram', 't_diagramlinks', 't_diagramobjects', 't_diagramtypes', 't_document', 't_ecf', 't_efforttypes', 't_files', 't_genopt', 't_glossary', 't_html', 't_image', 't_implement', 't_issues', 't_lists', 't_mainttypes', 't_method', 't_metrictypes', 't_object', 't_objectconstraint', 't_objecteffort', 't_objectfiles', 't_objectmetrics', 't_objectproblems', 't_objectproperties', 't_objectrequires', 't_objectresource', 't_objectrisks', 't_objectscenarios', 't_objecttests', 't_objecttrx', 't_objecttypes', 't_ocf', 't_operation', 't_operationparams', 't_operationposts', 't_operationpres', 't_operationtag', 't_package', 't_palette', 't_paletteitem', 't_phase', 't_primitives', 't_

In [34]:
# Maak een sessie
Session = sessionmaker(bind=engine)
session = Session()

try:
    # Verwijder records waar Property = 'name'
    session.query(ObjProp).filter(ObjProp.Property == 'name').delete(synchronize_session=False)
    session.commit()  # Wijzigingen opslaan
    print("Records succesvol verwijderd.")
except Exception as e:
    session.rollback()  # Terugdraaien bij een fout
    print(f"Fout bij het verwijderen van records: {e}")
finally:
    session.close()

Records succesvol verwijderd.


In [4]:
from sqlalchemy.orm import sessionmaker

# Maak een sessie
Session = sessionmaker(bind=engine)
session = Session()

# Query alle records uit XRef
records = session.query(Object).all()


# Omzetten naar DataFrame
data = [
    {column: getattr(record, column) for column in record.__table__.columns.keys()}
    for record in records
]
df = pd.DataFrame(data)

# Toon het DataFrame
#df[df['Client'].isin(['{440219A4-C64B-4eac-ADE5-E79ED6AA9BFE}', '{5772BEBB-97FA-42a9-B70D-DB55EAD6D1EE}', '{331C4A0B-1505-4945-A0B0-DCD8703AB50F}'])]
df

,Object_ID,Object_Type,Diagram_ID,Name,Alias,Author,Version,Note,Package_ID,Stereotype,...,IsRoot,IsLeaf,IsSpec,IsActive,StateFlags,PackageFlags,Multiplicity,StyleEx,ActionFlags,EventFlags
0,4,Package,0,Delfts Gemeentelijk Gegevensmodel,None,aashkpour en Arjen Brienen,2.2.0,None,2,Informatiemodel,...,0,0,0,0,None,None,None,None,None,None
1,5,Package,0,"0 Bestuur, Politiek en Ondersteuning",None,aashkpour,1.0,None,4,Domein,...,0,0,0,0,None,None,None,None,None,None
2,6,Package,0,Burgerzaken,None,Arjen Brienen,1.0,None,5,Domein,...,0,0,0,0,None,None,None,None,None,None
3,7,Package,0,Diagram,None,Arjen Brienen,1.0,None,6,None,...,0,0,0,0,None,None,None,None,None,None
4,8,Package,0,Model Burgerzaken,None,Arjen Brienen,1.0,None,6,None,...,0,0,0,0,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,2962,Class,0,RedenInstroom,None,arjen,1.2,De reden waarom de persoon de uitkering heeft gekregen.\n\nGeeft de reden van aanvraag van uitke...,74,Objecttype,...,0,0,0,0,None,None,None,None,None,None
1622,2966,Class,0,RedenBlokkering,None,arjen,1.2,"Als de dienst een uitkering betreft die periodiek wordt uitgekeerd, kan om redenen de betaling w...",74,Objecttype,...,0,0,0,0,None,None,None,None,None,None
1623,2968,Class,0,RedenUitstroom,None,arjen,1.2,De reden waarom de uitkering aan een persoon is beeindgd.\n\nReden toevoeging:\nGeeft de reden v...,74,Objecttype,...,0,0,0,0,None,None,None,None,None,None
1624,2970,Class,0,Huisvestingsoort,None,arjen,1.2,"Het type huisvesting van de client, zoals Huurder, Eigenaar, DakThuisloos. Nodig voor diepere an...",74,Objecttype,...,0,0,0,0,None,None,None,None,None,None


In [6]:
# Functie om het gewenste formaat te maken
def translate_eaid(value):
    # Verwijder 'EAID_' en vervang '_' door '-'
    core_value = value.replace("EAID_", "").replace("_", "-")
    # Omhul met accolades
    return f"{{{core_value}}}"


df_classes = pd.read_excel("../root/v2.2.0-develop/Gemeentelijk Gegevensmodel.xlsx", sheet_name="classes")
df_classes["ea_guid"] = df_classes["id"].apply(translate_eaid)

df_classes

,id,alias,archimate_type,authentiek,author,bron,created,datum_tijd_export,definitie,domein_dcat,...,phase,schema_id,status,stereotype,synoniemen,toelichting,uri,version,visibility,ea_guid
0,EAID_F1E55DC7_0F33_40ea_8713_2E1AC3D7EE8D,NaN,NaN,NaN,Arjen Brienen,NaN,2019-05-16 15:22:56,10-04-2024 17:26:26,iemand die meedoet aan eencollege- of raadsvergadering,NaN,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN,{F1E55DC7-0F33-40ea-8713-2E1AC3D7EE8D}
1,EAID_73FB5212_40ED_40dc_B837_36588840445A,NaN,NaN,NaN,Arjen Brienen,NaN,2019-05-16 11:55:17,10-04-2024 17:26:26,Een onderwerp dat in de vergadering wordt behandeld.,NaN,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN,{73FB5212-40ED-40dc-B837-36588840445A}
2,EAID_B9B82B25_5D7F_4d2b_84F9_4DF74525EEAD,NaN,NaN,NaN,Arjen Brienen,NaN,2019-05-16 11:44:01,28062023-11:06:06,Categorie waarop leveranciers zich voor de levering van personeel voor kunnen kwalificeren,NaN,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN,{B9B82B25-5D7F-4d2b-84F9-4DF74525EEAD}
3,EAID_7B9EDDFD_57F7_4ff2_938F_FDFA3B503DA8,NaN,NaN,NaN,Arjen Brienen,NaN,2019-05-16 11:26:04,10-04-2024 17:26:26,Iemand die behoort het college van burgemeester en wethouders,NaN,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN,{7B9EDDFD-57F7-4ff2-938F-FDFA3B503DA8}
4,EAID_475E24C9_CC9A_49cc_AB80_8CA0221441E5,NaN,NaN,NaN,Arjen Brienen,NaN,2019-05-16 11:44:26,10-04-2024 17:26:26,Samenhangende set gegevens en informatie voor een specifiek doel,NaN,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN,{475E24C9-CC9A-49cc-AB80-8CA0221441E5}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,EAID_27E447C9_4FFB_41db_B91D_95332B9E2249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,GGM_ROOT_SCHEMA,NaN,Objecttype,NaN,NaN,NaN,NaN,NaN,{27E447C9-4FFB-41db-B91D-95332B9E2249}
848,EAID_168B0065_09F2_4af5_8440_32AF28E9A309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,GGM_ROOT_SCHEMA,NaN,Objecttype,NaN,NaN,NaN,NaN,NaN,{168B0065-09F2-4af5-8440-32AF28E9A309}
849,EAID_E34C7512_03C1_443a_8716_E05BBAB8E873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,GGM_ROOT_SCHEMA,NaN,Objecttype,NaN,NaN,NaN,NaN,NaN,{E34C7512-03C1-443a-8716-E05BBAB8E873}
850,EAID_ECA02C0E_85B7_4e44_816C_E08092E01572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,GGM_ROOT_SCHEMA,NaN,Objecttype,NaN,NaN,NaN,NaN,NaN,{ECA02C0E-85B7-4e44-816C-E08092E01572}


In [7]:
pd.set_option('display.max_rows', 20)  # Toon alle rijen

# Merge/join op een gemeenschappelijke kolom
result = pd.merge(df, df_classes, on="ea_guid", how="inner")
print(result.columns)
result

Index(['Object_ID', 'Object_Type', 'Diagram_ID', 'Name', 'Alias', 'Author',
       'Version', 'Note', 'Package_ID', 'Stereotype', 'NType', 'Complexity',
       'Effort', 'Style', 'Backcolor', 'BorderStyle', 'BorderWidth',
       'Fontcolor', 'Bordercolor', 'CreatedDate', 'ModifiedDate', 'Status',
       'Abstract', 'Tagged', 'PDATA1', 'PDATA2', 'PDATA3', 'PDATA4', 'PDATA5',
       'Concurrency', 'Visibility', 'Persistence', 'Cardinality', 'GenType',
       'GenFile', 'Header1', 'Header2', 'Phase', 'Scope', 'GenOption',
       'GenLinks', 'Classifier', 'ea_guid', 'ParentID', 'RunState',
       'Classifier_guid', 'TPos', 'IsRoot', 'IsLeaf', 'IsSpec', 'IsActive',
       'StateFlags', 'PackageFlags', 'Multiplicity', 'StyleEx', 'ActionFlags',
       'EventFlags', 'id', 'alias', 'archimate_type', 'authentiek', 'author',
       'bron', 'created', 'datum_tijd_export', 'definitie', 'domein_dcat',
       'domein_iv3', 'gemma_definitie', 'gemma_naam', 'gemma_toelichting',
       'gemma_type', 'ge

,Object_ID,Object_Type,Diagram_ID,Name,Alias,Author,Version,Note,Package_ID,Stereotype,...,package_id,phase,schema_id,status,stereotype,synoniemen,toelichting,uri,version,visibility
0,12,Class,0,Aanwezige Deelnemer,None,Arjen Brienen,1.5,iemand die meedoet aan eencollege- of raadsvergadering,11,Objecttype,...,EAPK_9EBFD9E7_BB04_45c9_97BB_426207B40292,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN
1,13,Class,0,Agendapunt,None,Arjen Brienen,1.5,Een onderwerp dat in de vergadering wordt behandeld.,11,Objecttype,...,EAPK_9EBFD9E7_BB04_45c9_97BB_426207B40292,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN
2,14,Class,0,Categorie,None,Arjen Brienen,1.5,Categorie waarop leveranciers zich voor de levering van personeel voor kunnen kwalificeren,11,Objecttype,...,EAPK_9EBFD9E7_BB04_45c9_97BB_426207B40292,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN
3,15,Class,0,Collegelid,None,Arjen Brienen,1.5,Iemand die behoort het college van burgemeester en wethouders,11,Objecttype,...,EAPK_9EBFD9E7_BB04_45c9_97BB_426207B40292,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN
4,16,Class,0,Raadscommissie,None,Arjen Brienen,1.4,Een raadscommissie binnen de Nederlandse gemeenteraad is een groep raadsleden die zich buigt ove...,11,Objecttype,...,EAPK_9EBFD9E7_BB04_45c9_97BB_426207B40292,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,2946,Class,0,Intake,None,arjen,1.5,Dit is de fase tussen het eerste gesprek en het Plan van Aanpak. Tijdens de\nintakefase wordt ge...,65,Objecttype,...,EAPK_06C51790_1F81_4ac4_8E16_5177352EF2E1,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN
848,2962,Class,0,RedenInstroom,None,arjen,1.2,De reden waarom de persoon de uitkering heeft gekregen.\n\nGeeft de reden van aanvraag van uitke...,74,Objecttype,...,EAPK_740B2501_676F_4219_B57D_3C012B1AFC02,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,<memo>#NOTES#Een inhoudelijke toelichting op de toepassing van het informatie-element.,NaN,1.1,NaN
849,2966,Class,0,RedenBlokkering,None,arjen,1.2,"Als de dienst een uitkering betreft die periodiek wordt uitgekeerd, kan om redenen de betaling w...",74,Objecttype,...,EAPK_740B2501_676F_4219_B57D_3C012B1AFC02,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,<memo>#NOTES#Een inhoudelijke toelichting op de toepassing van het informatie-element.,NaN,1.1,NaN
850,2968,Class,0,RedenUitstroom,None,arjen,1.2,De reden waarom de uitkering aan een persoon is beeindgd.\n\nReden toevoeging:\nGeeft de reden v...,74,Objecttype,...,EAPK_740B2501_676F_4219_B57D_3C012B1AFC02,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,<memo>#NOTES#Een inhoudelijke toelichting op de toepassing van het informatie-element.,NaN,1.1,NaN


In [18]:
# Stap 1: Filter rijen waar de naam in hoofdletters staat
uppercase_rows = result['name'].str.isupper()

# Stap 2: Definieer een functie om CamelCase te maken
def to_camel_case(s):
    s = str(s)
    words = s.lower().split(' ')  # Verwerk als snake_case, voeg hier splitsing toe als nodig
    return ''.join(word.capitalize() for word in words)

# Stap 3: Pas CamelCase omvorming toe op de gefilterde rijen
#df.loc[uppercase_rows, 'name'] = df.loc[uppercase_rows, 'name'].apply(to_camel_case)

# Resultaat bekijken
result = result.dropna(subset=['name'])
result = result[result['name'].str.isupper()]
result['new_name'] = result['Name'].apply(to_camel_case)
result

,Object_ID,Object_Type,Diagram_ID,Name,Alias,Author,Version,Note,Package_ID,Stereotype,...,phase,schema_id,status,stereotype,synoniemen,toelichting,uri,version,visibility,new_name
16,38,Class,0,BOA,None,Arjen Brienen,1.5,Een buitengewoon opsporingsambtenaar (boa) is een ambtenaar met een specifieke opsporingsbevoegd...,15,Objecttype,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN,Boa
225,326,Class,0,B1,None,mkampen,1.2,None,62,Objecttype,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.2,NaN,B1
232,333,Class,0,KNM?,None,mkampen,1.3,None,62,Objecttype,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.3,NaN,Knm?
234,335,Class,0,MAP,None,mkampen,1.3,None,62,Objecttype,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.3,NaN,Map
237,338,Class,0,PIP,None,mkampen,1.3,None,62,Objecttype,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.3,NaN,Pip
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,1207,Class,0,PARTIJ,None,Haas_R,1.5,Een PARTIJ die bij de centrale voorzieningen van de BRP bekend is.,160,Objecttype,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,#NOTES#Het gaat hier om partijen die onder andere verantwoordelijk zijn van de bijhouding van ge...,NaN,1.4,NaN,Partij
749,1208,Class,0,REDEN VERKRIJGING NATIONALITEIT,None,Haas_R,1.5,Tabel waarin de redenen staan voor opname van de Nederlandse nationaliteit.,160,Objecttype,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN,RedenVerkrijgingNationaliteit
750,1209,Class,0,REDEN VERLIES NATIONALITEIT,None,Haas_R,1.5,Tabel waarin de redenen staan voor beëindiging van de Nederlandse nationaliteit.,160,Objecttype,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN,RedenVerliesNationaliteit
756,1215,Class,0,LAND,None,mante_h,1.5,"Een gedeelte van de wereld met een eigen bestuur, waarvan de soevereiniteit in ieder geval door ...",160,Objecttype,...,1.0,GGM_ROOT_SCHEMA,Proposed,Objecttype,NaN,NaN,NaN,1.4,NaN,Land


In [20]:
result[['Name', 'new_name', 'ea_guid']].to_excel("../output/output.xlsx") 

In [21]:
df_changes = pd.read_excel("../output/output.xlsx")

df_changes

,Unnamed: 0,Name,new_name,ea_guid
0,16,BOA,BOA,{90B2A249-8D88-4a14-979F-672223D98E8C}
1,225,B1,B1,{E5FC387A-F29B-460f-9811-3E6E1782B049}
2,232,KNM?,KNM?,{38CD50D1-D0D2-4550-BBAF-5FDC52C8109F}
3,234,MAP,MAP,{2177E7F0-0F76-41b7-B5CA-5FE182355E94}
4,237,PIP,PIP,{94EB7844-F863-4330-A15A-06ED4D1401E0}
...,...,...,...,...
67,748,PARTIJ,Partij,{0F175713-CEAB-4c6e-8A61-27F93C1FC777}
68,749,REDEN VERKRIJGING NATIONALITEIT,RedenVerkrijgingNationaliteit,{61D1E99C-DF5E-48ee-8BAB-4ACA1745BC7C}
69,750,REDEN VERLIES NATIONALITEIT,RedenVerliesNationaliteit,{7F97B43F-CCA1-4bfc-9824-9BEC8D2A70E6}
70,756,LAND,Land,{4B2CCD18-9004-4253-B72E-9243F3330F02}


In [22]:
from sqlalchemy.orm import sessionmaker

# Maak een sessie
Session = sessionmaker(bind=engine)
session = Session()


try:
    # Itereer over de DataFrame en update objecten
    for index, row in df_changes.iterrows():
        ea_guid = row['ea_guid']
        new_name = row['new_name']

        # Zoek het object met de bijbehorende ea_guid
        obj = session.query(Object).filter_by(ea_guid=ea_guid).first()

        if obj:
            obj.Name = new_name  # Update de naam
        else:
            print(f"Object met ea_guid {ea_guid} niet gevonden.")

    # Commit de wijzigingen
    session.commit()
    print(f"{len(df)} objecten succesvol bijgewerkt.")
except Exception as e:
    session.rollback()  # Herstel de transactie bij fouten
    print(f"Fout bij het bijwerken van objecten: {e}")
finally:
    session.close()

1626 objecten succesvol bijgewerkt.
